In [1]:
import pandas as pd
import numpy as np
import sklearn

print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)

/home/sidsharma/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.3.4
1.26.4
0.24.2


In [2]:
def load_data(path):
    data = pd.read_csv(path)
#     print(data.head())
#     print(data.shape)
#     data.info()
#     num_cols = data.select_dtypes(include=["int64", "float64"]).columns
#     cat_cols = data.select_dtypes(include=["object"]).columns
#     print("numerical col: ", num_cols)
#     print("categorical col: ", cat_cols)
    return data

train_data = load_data('train.csv')
test_data = load_data('test.csv')

In [3]:
train_data["Age"].describe()
# print(train_data["Age"].isnull().sum())

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [4]:
# train_data["Embarked"].describe()

In [5]:
age_median = train_data["Age"].median()
print("age_median", age_median)
fare_median = train_data["Fare"].median()
print("fare_median", fare_median)
embark_mode = train_data["Embarked"].mode()
print("embark_mode", embark_mode)

age_median 28.0
fare_median 14.4542
embark_mode 0    S
dtype: object


In [6]:
train_data["Age"].fillna(age_median, inplace=True)
train_data["Fare"].fillna(fare_median, inplace=True)
train_data["Embarked"].fillna(embark_mode[0], inplace=True)

In [7]:
# train_data["Age"].describe()

In [8]:
# train_data["Fare"].describe()

In [9]:
# train_data["Embarked"].describe()

In [10]:
# print(train_data[["Age","Fare","Embarked"]].isnull().sum())

In [11]:
# print(test_data[["Age","Fare","Embarked"]].isnull().sum())

In [12]:
test_data["Age"].fillna(age_median, inplace=True)
test_data["Fare"].fillna(fare_median, inplace=True)
test_data["Embarked"].fillna(embark_mode[0], inplace=True)

In [13]:
print(test_data[["Age","Fare","Embarked"]].isnull().sum())

Age         0
Fare        0
Embarked    0
dtype: int64


In [14]:
m = {'male': 0, 'female': 1}
print(m)

{'male': 0, 'female': 1}


In [15]:
train_data["sex_bin"] = train_data["Sex"].map(m)

In [16]:
train_data[["Sex", "sex_bin"]].head()

,Sex,sex_bin
0,male,0
1,female,1
2,female,1
3,female,1
4,male,0


In [17]:
test_data["sex_bin"] = test_data["Sex"].map(m)

In [18]:
test_data[["Sex", "sex_bin"]].head()

,Sex,sex_bin
0,male,0
1,female,1
2,male,0
3,male,0
4,female,1


In [19]:
full_data = pd.concat([train_data, test_data], ignore_index=True)
print(full_data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'sex_bin'],
      dtype='object')


In [20]:
full_data.shape

(1309, 13)

In [21]:
full_data["Survived"].isnull().sum()

418

In [22]:
embark_series = pd.get_dummies(full_data["Embarked"], prefix="Embarked")

In [23]:
full_data = pd.concat([full_data, embark_series], axis=1)

In [24]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1309 non-null   object 
 12  sex_bin      1309 non-null   int64  
 13  Embarked_C   1309 non-null   uint8  
 14  Embarked_Q   1309 non-null   uint8  
 15  Embarked_S   1309 non-null   uint8  
dtypes: float64(3), int64(5), object(5), uint8(3)
memory usage: 136.9+ KB


In [25]:
full_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'sex_bin', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [26]:
full_data = full_data.drop(columns=["Embarked"])

In [27]:
full_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'sex_bin', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [28]:
family_data = pd.DataFrame({"FamilySize": full_data["SibSp"]+full_data["Parch"]+1})
family_data.head()

,FamilySize
0,2
1,2
2,1
3,2
4,1


In [29]:
isalone_data = pd.DataFrame({"IsAlone": family_data["FamilySize"] == 1}).astype(int)
print(isalone_data.head())

   IsAlone
0        0
1        0
2        1
3        0
4        1


In [30]:
ischild_data = pd.DataFrame({"IsChild": full_data["Age"] < 16}).astype(int)
print(ischild_data.head())

   IsChild
0        0
1        0
2        0
3        0
4        0


In [31]:
full_data = pd.concat([full_data, family_data, isalone_data, ischild_data], axis=1)
full_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,sex_bin,Embarked_C,Embarked_Q,Embarked_S,FamilySize,IsAlone,IsChild
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,0,0,0,1,2,0,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,1,1,0,0,2,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0,1,1,1,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,1,0,0,1,2,0,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,0,0,0,1,1,1,0


In [32]:
train_data, test_data = full_data[:train_data.shape[0]], full_data[train_data.shape[0]:]

In [33]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  sex_bin      891 non-null    int64  
 12  Embarked_C   891 non-null    uint8  
 13  Embarked_Q   891 non-null    uint8  
 14  Embarked_S   891 non-null    uint8  
 15  FamilySize   891 non-null    int64  
 16  IsAlone      891 non-null    int64  
 17  IsChild      891 non-null    int64  
dtypes: float64(3), int64(8), object(4), uint8(3)
memor

In [34]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 891 to 1308
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     0 non-null      float64
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          418 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         418 non-null    float64
 10  Cabin        91 non-null     object 
 11  sex_bin      418 non-null    int64  
 12  Embarked_C   418 non-null    uint8  
 13  Embarked_Q   418 non-null    uint8  
 14  Embarked_S   418 non-null    uint8  
 15  FamilySize   418 non-null    int64  
 16  IsAlone      418 non-null    int64  
 17  IsChild      418 non-null    int64  
dtypes: float64(3), int64(8), object(4), uint8(3)
me

In [35]:
train_data, test_data = full_data[:train_data.shape[0]], full_data[train_data.shape[0]:]
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  sex_bin      891 non-null    int64  
 12  Embarked_C   891 non-null    uint8  
 13  Embarked_Q   891 non-null    uint8  
 14  Embarked_S   891 non-null    uint8  
 15  FamilySize   891 non-null    int64  
 16  IsAlone      891 non-null    int64  
 17  IsChild      891 non-null    int64  
dtypes: float64(3), int64(8), object(4), uint8(3)
memor

In [36]:
train_data = train_data.drop(columns=["Name", "Sex", "Ticket", "Cabin", "SibSp", "Parch"])
test_data  = test_data.drop(columns=["Name", "Sex", "Ticket", "Cabin", "SibSp", "Parch"])

In [37]:
x_train = train_data[["Pclass","Age","Fare","sex_bin","Embarked_C","Embarked_Q","Embarked_S","FamilySize","IsAlone","IsChild"]]

In [38]:
y_train = train_data["Survived"]

In [39]:
x_test = test_data[["Pclass","Age","Fare","sex_bin","Embarked_C","Embarked_Q","Embarked_S","FamilySize","IsAlone","IsChild"]]

In [40]:
print(x_train.shape, y_train.shape, x_test.shape)

(891, 10) (891,) (418, 10)


In [41]:
from sklearn.linear_model import LogisticRegression as LR

In [42]:
max_iter = 500
# for C in [0.01, 0.1, 1, 10, 100]:
model = LR(max_iter=max_iter)
model.fit(x_train, y_train)
print(model.score(x_train, y_train))

0.8092031425364759


In [43]:
y_test = model.predict(x_test)

In [44]:
y_test.shape

(418,)

In [45]:
final_df = pd.DataFrame({"PassengerId":test_data["PassengerId"], "Survived":y_test})

In [46]:
final_df.shape
final_df.head()

,PassengerId,Survived
891,892,0.0
892,893,1.0
893,894,0.0
894,895,0.0
895,896,1.0


In [47]:
final_df["Survived"] = final_df["Survived"].astype(int)

In [48]:
final_df.reset_index(drop=True, inplace=True)

In [49]:
final_df.to_csv("./submission_03022026_iter3.csv", index=False)